In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/keypoints_final_256.csv')
df_test = pd.read_csv('/content/drive/MyDrive/keypoints_final_test_256.csv')

In [ ]:
df = df.drop('Unnamed: 0',1)
df_test = df_test.drop('Unnamed: 0',1)

<ipython-input-79-2408e182c2cf>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Unnamed: 0',1)
<ipython-input-79-2408e182c2cf>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_test = df_test.drop('Unnamed: 0',1)


In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,1.1
0,0.561012,0.302157,-0.123826,0.564638,0.289460,-0.105465,0.567299,0.289761,-0.105632,0.569772,...,0.533098,0.883884,0.116445,0.566635,0.933335,0.036987,0.536138,0.938561,-0.000178,tadasana
1,0.561800,0.300378,-0.112502,0.564178,0.286860,-0.094147,0.566694,0.287539,-0.094309,0.568935,...,0.532270,0.883887,0.118631,0.570173,0.931963,0.035905,0.532538,0.933740,-0.001898,tadasana
2,0.561128,0.303233,-0.398081,0.564855,0.286085,-0.388905,0.567303,0.285346,-0.389124,0.569531,...,0.527861,0.878289,0.133518,0.566152,0.918341,0.058699,0.534654,0.928207,-0.001211,tadasana
3,0.560880,0.309663,-0.300727,0.564582,0.291333,-0.289611,0.567254,0.290768,-0.289883,0.570002,...,0.536183,0.912010,0.188811,0.567818,0.914286,0.152388,0.530323,0.931029,0.077729,tadasana
4,0.561824,0.313649,-0.342597,0.565599,0.294846,-0.333475,0.568605,0.294382,-0.333750,0.571083,...,0.540910,0.887583,0.192771,0.571397,0.923574,0.116451,0.531012,0.930395,0.070042,tadasana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13124,0.486648,0.286260,-0.089110,0.490879,0.273558,-0.067470,0.493832,0.272559,-0.067595,0.497064,...,0.470475,0.872472,0.102894,0.496999,0.912968,0.022522,0.466042,0.915865,-0.025367,tadasana
13125,0.482443,0.300422,-0.362747,0.487832,0.286010,-0.350695,0.491121,0.285521,-0.350950,0.493022,...,0.474411,0.877378,0.126974,0.494784,0.912487,0.072589,0.465412,0.914555,0.006590,tadasana
13126,0.480868,0.294958,-0.321305,0.485442,0.280226,-0.312262,0.488200,0.280182,-0.312505,0.491096,...,0.473303,0.876180,0.188003,0.495985,0.914854,0.099370,0.465068,0.916696,0.075990,tadasana
13127,0.476346,0.293068,-0.347669,0.480525,0.276444,-0.338560,0.483666,0.276016,-0.338818,0.485902,...,0.473899,0.877159,0.200288,0.494451,0.915021,0.085202,0.464427,0.913910,0.084059,tadasana


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder

In [ ]:
lbe = LabelEncoder()
df['1.1']= lbe.fit_transform(df['1.1'])
df_test['1.1']= lbe.fit_transform(df_test['1.1'])
dict(zip(lbe.classes_, lbe.transform(lbe.classes_)))

{'bhujan': 0, 'padmasan': 1, 'shav': 2, 'tadasana': 3, 'trik': 4, 'vriksh': 5}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class YogaDataset(Dataset):
  def __init__(self, df):
    self.features = df.drop(['1.1'], axis=1)
    self.target = df['1.1']

  def __len__(self):
    return len(self.features)
  
  def __getitem__(self, index):
    features = self.features.loc[index]
    target = self.target[index]
    return torch.tensor(features.tolist()).float().to(device), torch.tensor(target).long().to(device)

In [ ]:
# df_feat = df.drop(['1.1'], axis=1)

In [ ]:
device

'cpu'

In [ ]:
# target = df['1.1']

In [ ]:
data = YogaDataset(df)

In [ ]:
feat, target = data[100]

In [ ]:
target

tensor(3)

In [ ]:
class RNN(nn.Module):
  def __init__(self,hidden_dim=20, input_dim=99, sequence_num=16, n_layers=1):
    super(RNN, self).__init__()
    self.input_dim = input_dim
    self.sequence_num = sequence_num
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
    self.rnn = nn.RNN(input_dim,hidden_dim, n_layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, 6)
  def forward(self, input):
    h0 = torch.zeros(self.n_layers,  self.hidden_dim)
    out, _ = self.rnn(input,h0)
    pred = self.fc(out)
    output = nn.Softmax( dim=1)(pred)
    return output

In [ ]:
model = RNN().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
model

RNN(
  (rnn): RNN(99, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=6, bias=True)
)

In [ ]:
def get_data():
    train = YogaDataset(df)
    test = YogaDataset(df_test)
    trn_dl = DataLoader(train, batch_size=16,shuffle=True, drop_last=True)
    test_dl = DataLoader(test, batch_size=16, shuffle=True, drop_last=True)
    return trn_dl, test_dl

In [ ]:
trn_ldr, test_ldr = get_data()
# for x in trn_ldr:
#   print(x)
#   break

In [ ]:
def train_batch(x, y, model, loss_fn, opt):
    model.train()
    prediction = model(x)
    # print(y.shape)
    # print(y)
    # print(prediction.shape)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [ ]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    pred = model(x)
    _, is_correct = torch.max(pred.data,1)
    acc_all = (is_correct==y).detach().cpu().numpy()
    return acc_all

In [ ]:
@torch.no_grad()
def val_loss_trn(x, y, model, loss_fn):
  prediction = model(x)
  val_loss = loss_fn(prediction, y)
  return val_loss.item()

In [ ]:
train_loss = []
train_accuracies= []
val_loss = [] 
val_accuracies = []

for i in range(10):
    print(f'Epoch: _________*****{i}*****_______')
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies, val_epoch_losses = [], []



    for ix, batch in (enumerate(iter(trn_ldr))):

        x, y = batch
        train_epoch_losses.append(train_batch(x, y, model, loss_fn, optimizer))
    train_epoch_loss = np.array(train_epoch_losses).mean()
    print(f'Epoch: _________*****{i} Training Loss : {train_epoch_loss} *****_______')



    for ix, batch in (enumerate(iter(trn_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        train_epoch_accuracies.append(sum(accuracy(x, y, model)) / len(y))
    train_epoch_accuracy = np.array(train_epoch_accuracies).mean()
    print(f'Epoch: _________*****{i} Training Accuracy: {train_epoch_accuracy} *****_______')


    for ix, batch in (enumerate(iter(test_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        val_epoch_losses.append(val_loss_trn(x, y, model, loss_fn))
    val_epoch_loss = np.array(val_epoch_losses).mean()
    print(f'Epoch: _________*****{i} Validation Loss : {val_epoch_loss} *****_______')

    for ix, batch in (enumerate(iter(test_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        val_epoch_accuracies.append(sum(accuracy(x, y, model)) / len(y))
    val_epoch_accuracy = np.array(val_epoch_accuracies).mean()
    print(f'Epoch: _________*****{i} Validation Accuracy. {val_epoch_accuracy} *****_______')
    print('\n')

    train_loss.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)
    val_loss.append(val_epoch_loss)

Epoch: _________*****0*****_______
Epoch: _________*****0 Training Loss : 1.3545948684215545 *****_______
Epoch: _________*****0 Training Accuracy: 0.9480182926829268 *****_______
Epoch: _________*****0 Validation Loss : 1.1210019684004036 *****_______
Epoch: _________*****0 Validation Accuracy. 0.9666811846689896 *****_______


Epoch: _________*****1*****_______
Epoch: _________*****1 Training Loss : 1.116492057864259 *****_______
Epoch: _________*****1 Training Accuracy: 0.9556402439024391 *****_______
Epoch: _________*****1 Validation Loss : 1.0802388963799028 *****_______
Epoch: _________*****1 Validation Accuracy. 0.98301393728223 *****_______


Epoch: _________*****2*****_______
Epoch: _________*****2 Training Loss : 1.0952995152008242 *****_______
Epoch: _________*****2 Training Accuracy: 0.9614329268292683 *****_______
Epoch: _________*****2 Validation Loss : 1.0852552743738952 *****_______
Epoch: _________*****2 Validation Accuracy. 0.9671167247386759 *****_______


Epoch: ___

In [ ]:
df_val = pd.read_csv('/content/drive/MyDrive/keypoints_final_val_256.csv').drop('Unnamed: 0',1)

<ipython-input-106-a67a6a83625a>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_val = pd.read_csv('/content/drive/MyDrive/keypoints_final_val_256.csv').drop('Unnamed: 0',1)


In [ ]:
df_val[512:(512+256)]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,1.1
512,0.498204,0.325045,-0.280092,0.501241,0.313319,-0.268156,0.504362,0.314049,-0.268400,0.506766,...,0.471870,0.837673,0.210182,0.500933,0.863186,0.104345,0.443333,0.851790,0.119382,vriksh
513,0.497350,0.325974,-0.288618,0.500722,0.314890,-0.276627,0.503065,0.314921,-0.276867,0.505141,...,0.472278,0.835503,0.208577,0.500395,0.863170,0.087212,0.443110,0.854528,0.116487,vriksh
514,0.495267,0.325460,-0.297356,0.498773,0.313994,-0.284876,0.501191,0.314042,-0.285120,0.503176,...,0.471952,0.835442,0.214445,0.499915,0.863171,0.099858,0.443945,0.853825,0.122538,vriksh
515,0.491493,0.325750,-0.294950,0.494320,0.313288,-0.282730,0.497406,0.313282,-0.282968,0.499448,...,0.472206,0.835779,0.202842,0.499877,0.863686,0.083113,0.443558,0.854689,0.108751,vriksh
516,0.489973,0.323742,-0.254229,0.493444,0.311668,-0.241386,0.495739,0.311929,-0.241621,0.498364,...,0.472216,0.835538,0.191708,0.498591,0.861988,0.105938,0.445896,0.852941,0.097599,vriksh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0.467412,0.332124,-0.161435,0.471836,0.319425,-0.143835,0.474202,0.319999,-0.143992,0.476708,...,0.475597,0.831335,0.103849,0.472315,0.709288,0.044309,0.463503,0.858247,0.023949,vriksh
764,0.468211,0.333263,-0.166658,0.472588,0.320537,-0.149037,0.474694,0.320821,-0.149193,0.477804,...,0.476207,0.828265,0.104060,0.471883,0.710419,0.036686,0.464533,0.858780,0.024493,vriksh
765,0.469188,0.333790,-0.164722,0.473254,0.320903,-0.147057,0.475751,0.320856,-0.147215,0.479065,...,0.475647,0.828216,0.094975,0.471934,0.709247,0.044217,0.465466,0.859077,0.015660,vriksh
766,0.469656,0.332489,-0.155165,0.473491,0.319844,-0.137579,0.476636,0.319893,-0.137735,0.479524,...,0.476204,0.829023,0.107831,0.472267,0.711030,0.044306,0.465084,0.859327,0.029730,vriksh


In [ ]:
tad =  df_val[512:512+256]
tad= tad.reset_index( drop=True)
# tad_feat = tad.drop('1.1',1)

In [ ]:
class YogaDataset(Dataset):
  def __init__(self, df):
    self.features = df.drop(['1.1'], axis=1)
    self.target = df['1.1']

  def __len__(self):
    return len(self.features)
  
  def __getitem__(self, index):
    features = self.features.loc[index]
    target = self.target[index]
    return torch.tensor(features.tolist()).float().to(device)

In [ ]:
tad = YogaDataset(tad)
tad_dl = DataLoader(tad, batch_size=16,drop_last=False)

In [ ]:
for batch in tad_dl:
  
  pred=(model(batch))
  _, is_correct = torch.max(pred.data,1)
  print(is_correct)

tensor([4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])


In [ ]:
{'bhujan': 0, 'padmasan': 1, 'shav': 2, 'tadasana': 3, 'trik': 4, 'vriksh': 5}